# Byte-Pair encoding

**byte-pair encoding algorithm** is a simple data compression technique that iteratively replaces the most frequent pair of bytes in a sequence with a single, unused byte. It is often used as a sub-step in more complex algorithms, such as in **sentencepiece**.

first, the corpus needs to be made. This is done using iteratively applying a process_file function to each file name in a list of file names.

In [36]:
import re
import yaml
from pathlib import Path
import glob
from nltk.tokenize import RegexpTokenizer

corpus_dir = Path('data') 
document_paths = glob.glob(str(corpus_dir / '*.txt')) 

def process_document(file_path):
    with open(file_path, 'r') as f:
        data = f.read()
        
    yaml_header = re.search(r'---\n(.*?)\n---', data, re.DOTALL)
    metadata = yaml.safe_load(yaml_header.group(1))  # Parse YAML into a dictionary

    # Remove YAML frontmatter from the text
    text_without_yaml = re.sub(r'---\n(.*?)\n---', '', data, flags=re.DOTALL)

    tokens = RegexpTokenizer(r'\w+').tokenize(text_without_yaml)
    tokens = [t.lower() for t in tokens]
    
    document = dict(metadata=metadata, text=text_without_yaml, ws_tokens=tokens)
    
    return document

corpus = dict()

for path in document_paths:
    document = process_document(path)
    corpus[document['metadata']['title']] = document

In [38]:
for document in corpus:
    for key in corpus[document]['metadata']:
        print(f"{key} : {corpus[document]['metadata'][key]}")
        
    print(f"first 100 out of {len(corpus[document]['text'])} characters: {corpus[document]['text'][:100]}")
    print(f"first 30 out of {len(corpus[document]['ws_tokens'])} ws_tokens: {corpus[document]['ws_tokens'][:30]}")
    print()

title : Archeological Dig at Old Montreal Hospital on Hold by McGill University
author : Emelia Fournier
publisher : aptn news
URL : https://www.aptnnews.ca/national-news/archeological-dig-old-montreal-hospital-on-hold-mcgill-university/
summary : This article reports on the temporary halt of an archeological dig at an old Montreal hospital by McGill University.
tags : ['news', 'indigenous']
first 100 out of 5136 characters: 
A spokesperson for the Mohawk Mothers, or Kahnistensera, says the group feels pushed aside in the s
first 30 out of 877 ws_tokens: ['a', 'spokesperson', 'for', 'the', 'mohawk', 'mothers', 'or', 'kahnistensera', 'says', 'the', 'group', 'feels', 'pushed', 'aside', 'in', 'the', 'search', 'for', 'unmarked', 'graves', 'on', 'a', 'site', 'owned', 'by', 'société', 'québécoise', 'des', 'infrastructures', 'or']

title : mcgill-reports-nine-potential-grave-zones-at-new-vic-site-a-week-after-security-verbally-assaulted-mohawk-mothers
author : jasjot grewal
publisher : the tr

In [52]:
## how many ws words
vocab_set = Counter()
for document in corpus:
    vocab = Counter(corpus[document]['ws_tokens'])
    vocab_set = set(vocab)
    print(f"length of vocab: {len(vocab_set)}")

length of vocab: 361
length of vocab: 511
length of vocab: 196
length of vocab: 432
length of vocab: 191
length of vocab: 365
length of vocab: 243
length of vocab: 217
length of vocab: 304
length of vocab: 391
length of vocab: 304
length of vocab: 178
length of vocab: 151
length of vocab: 151
length of vocab: 395
length of vocab: 57
length of vocab: 335
length of vocab: 566
length of vocab: 320
length of vocab: 161
length of vocab: 258
length of vocab: 246
length of vocab: 1074
length of vocab: 325
length of vocab: 125
length of vocab: 395
length of vocab: 418
length of vocab: 486
length of vocab: 233
length of vocab: 454
length of vocab: 318
length of vocab: 290
length of vocab: 188
length of vocab: 267
length of vocab: 247
length of vocab: 492
length of vocab: 319
length of vocab: 145
length of vocab: 311
length of vocab: 308
length of vocab: 174
length of vocab: 331


In [ ]:
import re, collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def byte_pair_tokenizer(string, vocab_size):
    # Initialize vocabulary
    vocab = collections.defaultdict(int)
    words = string.split()
    for word in words:
        vocab[' '.join(list(word)) + ' </w>'] = 1

    for i in range(vocab_size):
        pairs = get_stats(vocab)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        vocab = merge_vocab(best, vocab)

    # Tokenize the string
    tokens = [word for word in vocab.keys()]
    tokens = [re.sub(r' </w>', '', token) for token in tokens]

    return tokens

# Test the tokenizer
tokens = byte_pair_tokenizer("Hello, world!", 100)
print(tokens)

In [53]:
all_tokens = []
for document in corpus:
    all_tokens += corpus[document]['ws_tokens']

In [56]:
all_tokens = [t.lower() for t in all_tokens]
print(f"num tokens: {len(all_tokens)} \n num unique tokens: {len(set(all_tokens))} \n first 100 tokens: {all_tokens[:100]}")

num tokens: 29843 
 num unique tokens: 3793 
 first 100 tokens: ['a', 'spokesperson', 'for', 'the', 'mohawk', 'mothers', 'or', 'kahnistensera', 'says', 'the', 'group', 'feels', 'pushed', 'aside', 'in', 'the', 'search', 'for', 'unmarked', 'graves', 'on', 'a', 'site', 'owned', 'by', 'société', 'québécoise', 'des', 'infrastructures', 'or', 'sqi', 'mcgill', 'says', 'it', 'leases', 'part', 'of', 'the', 'property', 'the', 'process', 'can', 'no', 'longer', 'by', 'any', 'means', 'be', 'considered', 'indigenous', 'led', 'as', 'the', 'sqi', 'and', 'mcgill', 'attempt', 'to', 'control', 'the', 'whole', 'process', 'reducing', 'the', 'role', 'of', 'indigenous', 'people', 'to', 'performing', 'ceremonies', 'on', 'the', 'site', 'said', 'kahentinetha', 'one', 'of', 'the', 'mothers', 'who', 'added', 'that', 'they', 'feel', 'blindsided', 'by', 'the', 'communications', 'that', 'happened', 'without', 'consulting', 'them', 'quebec', 's', 'infrastructure', 'society', 'or', 'sqi']


In [58]:
string = " ".join(all_tokens)
print(f"first 100 characters: {string[:100]}")

first 100 characters: a spokesperson for the mohawk mothers or kahnistensera says the group feels pushed aside in the sear


In [73]:
import re, collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def byte_pair_tokenizer(string, vocab_size):
    # Initialize vocabulary
    vocab = collections.defaultdict(int)
    words = string.split()
    for word in words:
        vocab[' '.join(list(word)) + ' </w>'] = 1

    for i in range(vocab_size):
        pairs = get_stats(vocab)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        vocab = merge_vocab(best, vocab)

    # Tokenize the string
    tokens = [word for word in vocab.keys()]
    tokens = [re.sub(r' </w>', '', token) for token in tokens]

    return tokens

# Test the tokenizer
tokens = byte_pair_tokenizer(string, 4000)
print(tokens)

['a</w>', 'spokesperson</w>', 'for</w>', 'the</w>', 'mohawk</w>', 'mothers</w>', 'or</w>', 'kahnistensera</w>', 'says</w>', 'group</w>', 'feels</w>', 'pushed</w>', 'aside</w>', 'in</w>', 'search</w>', 'unmarked</w>', 'graves</w>', 'on</w>', 'site</w>', 'owned</w>', 'by</w>', 'société</w>', 'québécoise</w>', 'des</w>', 'infrastructures</w>', 'sqi</w>', 'mcgill</w>', 'it</w>', 'leases</w>', 'part</w>', 'of</w>', 'property</w>', 'process</w>', 'can</w>', 'no</w>', 'longer</w>', 'any</w>', 'means</w>', 'be</w>', 'considered</w>', 'indigenous</w>', 'led</w>', 'as</w>', 'and</w>', 'attempt</w>', 'to</w>', 'control</w>', 'whole</w>', 'reducing</w>', 'role</w>', 'people</w>', 'performing</w>', 'ceremonies</w>', 'said</w>', 'kahentinetha</w>', 'one</w>', 'who</w>', 'added</w>', 'that</w>', 'they</w>', 'feel</w>', 'blindsided</w>', 'communications</w>', 'happened</w>', 'without</w>', 'consulting</w>', 'them</w>', 'quebec</w>', 's</w>', 'infrastructure</w>', 'society</w>', 'both</w>', 'put</w>', 

In [79]:
import re, collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def byte_pair_tokenizer(string, vocab_size, lexicon):
    # Initialize vocabulary
    vocab = collections.defaultdict(int)
    words = string.split()
    for word in words:
        vocab[' '.join(list(word)) + ' </w>'] = 1

    # Add entities and morphemes from the lexicon to the vocabulary
    for category in lexicon.values():
        for item in category:
            vocab[' '.join(list(item)) + ' </w>'] = 1

    for i in range(vocab_size):
        pairs = get_stats(vocab)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        vocab = merge_vocab(best, vocab)

    # Tokenize the string
    tokens = [word for word in vocab.keys()]
    tokens = [re.sub(r' </w>', '', token) for token in tokens]

    return tokens

# Define a lexicon
lexicon = {
    'entities': ['the mowhawk mothers</w>', 'khanistensera'],
    'morphemes': ['ing', 'ed', 's', 'ness', 'less', 'ful', 'ly', "n't", "'m", "'ll", "'s", "'ve", "'re", "'d"]
}

# Test the tokenizer
tokens = byte_pair_tokenizer(string, 3000, lexicon)
print(tokens)

['a</w>', 'spokesperson</w>', 'for</w>', 'the</w>', 'mohawk</w>', 'mothers</w>', 'or</w>', 'kahnistensera</w>', 'says</w>', 'group</w>', 'feels</w>', 'pushed</w>', 'aside</w>', 'in</w>', 'search</w>', 'unmarked</w>', 'graves</w>', 'on</w>', 'site</w>', 'owned</w>', 'by</w>', 'société</w>', 'québécoise</w>', 'des</w>', 'infrastructures</w>', 'sqi</w>', 'mcgill</w>', 'it</w>', 'leases</w>', 'part</w>', 'of</w>', 'property</w>', 'process</w>', 'can</w>', 'no</w>', 'longer</w>', 'any</w>', 'means</w>', 'be</w>', 'considered</w>', 'indigenous</w>', 'led</w>', 'as</w>', 'and</w>', 'attempt</w>', 'to</w>', 'control</w>', 'whole</w>', 'reducing</w>', 'role</w>', 'people</w>', 'performing</w>', 'ceremonies</w>', 'said</w>', 'kahentinetha</w>', 'one</w>', 'who</w>', 'added</w>', 'that</w>', 'they</w>', 'feel</w>', 'blindsided</w>', 'communications</w>', 'happened</w>', 'without</w>', 'consulting</w>', 'them</w>', 'quebec</w>', 's</w>', 'infrastructure</w>', 'society</w>', 'both</w>', 'put</w>', 

In [82]:
import re, collections

def byte_pair_tokenizer(string, vocab_size, lexicon):
    # Initialize vocabulary
    vocab = collections.defaultdict(int)
    words = string.split()
    for word in words:
        vocab[' '.join(list(word)) + ' </w>'] = 1

    # Add entities and morphemes from the lexicon to the vocabulary
    for category in lexicon.values():
        for item in category:
            if ' ' in item:  # Treat entities as single words
                vocab[item + ' </w>'] = 1
            else:
                vocab[' '.join(list(item)) + ' </w>'] = 1

    for i in range(vocab_size):
        pairs = get_stats(vocab)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        vocab = merge_vocab(best, vocab)

    # Tokenize the string
    tokens = [word for word in vocab.keys()]
    tokens = [re.sub(r' </w>', '', token) for token in tokens]

    return tokens

# Test the tokenizer
tokens = byte_pair_tokenizer(string, 300, lexicon)
print(tokens)

['a</w>', 's po k es per s on</w>', 'for', 'th e</w>', 'mo h aw k</w>', 'mo th ers</w>', 'or</w>', 'ka h n ist en ser a</w>', 's ay s</w>', 'g ro u p</w>', 'fe el s</w>', 'p us h ed</w>', 'a sid e</w>', 'in</w>', 'se arch', 'un mar ked</w>', 'g ra ves</w>', 'on</w>', 'si te</w>', 'ow n ed</w>', 'b y</w>', 'so ci é t é', 'qu é b é co is e</w>', 'd es</w>', 'in f ra struc tu res</w>', 's q i', 'm c g il l</w>', 'it</w>', 'le as es</w>', 'par t</w>', 'o f</w>', 'pro per ty</w>', 'pro c ess</w>', 'c an</w>', 'no', 'l on g er</w>', 'an y</w>', 'me ans</w>', 'b e</w>', 'con si der ed</w>', 'in di gen ous</w>', 'l ed</w>', 'a s</w>', 'and</w>', 'att em p t</w>', 'to', 'cont ro l</w>', 'wh ol e</w>', 're duc ing</w>', 'ro le</w>', 'pe op le</w>', 'per form ing</w>', 'c er em on ies</w>', 'sa i d</w>', 'ka h ent in e th a</w>', 'one</w>', 'w ho', 'ad ded</w>', 'th a t</w>', 'th e y</w>', 'fe el</w>', 'bl in d si ded</w>', 'comm un ic ations</w>', 'h ap pen ed</w>', 'w it h ou t</w>', 'con s ul 

In [75]:
decoded_tokens = [t[:-4] if t.endswith('</w>') else t for t in tokens]
decoded_tokens

['a',
 'spokesperson',
 'for',
 'the',
 'mohawk',
 'mothers',
 'or',
 'kahnistensera',
 'says',
 'group',
 'feels',
 'pushed',
 'aside',
 'in',
 'search',
 'unmarked',
 'graves',
 'on',
 'site',
 'owned',
 'by',
 'société',
 'québécoise',
 'des',
 'infrastructures',
 'sqi',
 'mcgill',
 'it',
 'leases',
 'part',
 'of',
 'property',
 'process',
 'can',
 'no',
 'longer',
 'any',
 'means',
 'be',
 'considered',
 'indigenous',
 'led',
 'as',
 'and',
 'attempt',
 'to',
 'control',
 'whole',
 'reducing',
 'role',
 'people',
 'performing',
 'ceremonies',
 'said',
 'kahentinetha',
 'one',
 'who',
 'added',
 'that',
 'they',
 'feel',
 'blindsided',
 'communications',
 'happened',
 'without',
 'consulting',
 'them',
 'quebec',
 's',
 'infrastructure',
 'society',
 'both',
 'put',
 'out',
 'statements',
 'aug',
 '3',
 'saying',
 'nine',
 'potential',
 'gravesites',
 'were',
 'identified',
 'through',
 'ground',
 'penetrating',
 'radar',
 'gpr',
 'excluded',
 'information',
 'according',
 'geoscan'